<a href="https://colab.research.google.com/github/boyerb/Investments/blob/master/Ex07-WRDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Investments: Theory and Data Analysis**, Bates, Boyer, and Fletcher

# Example Chapter 7: The WRDS API
In this example we illustrate how to access CRSP data on equities using the WRDS API. The script uses the function `get_crsp_msf_by_ids` to download monthly data where `msf` stands for *monthly stock file*. A similar function, `get_crsp_dsf_by_ids`, allows you to access daily data where `dsf` stands for *daily stock file*. These functions are part of the `simple_finance` package.

To use this script below, you will first need to obtain a WRDS account through your institution with a username and password, and set up dual authentication through the WRDS site.

###Imports and Setup
Below we install and import a variety of packages that provide useful functions. The `simple_finance` and `wrds` packages are not included in Colab by default and require installation.

To improve readability when printing a DataFrame to the consol, we then configure Pandas display options: (1) show all columns without truncation, (2) widen the output to 1000 characters so rows print on a single line, and (3) truncate long tables.

In [ ]:
# Install Packages
%pip install -q simple-finance
import simple_finance as sf
%pip -q install wrds
import wrds
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Establish display options when printing DataFrames
pd.set_option('display.max_columns', None)   # Show all columns without truncation
pd.set_option('display.width', 1000)   # Set the display width so output stays on one line
pd.set_option("display.max_rows", 20) # Force truncation if DataFrame has more than 20 rows

### Establish a WRDS Connection
When you run the command below you will be prompted to enter your WRDS username and password, and then complete dual authentication on your phone. You will then be asked if you want to create a `.pgpass` file. In local environments such as PyCharm or Visual Studio, a `.pgpass` file stored on your PC allows you to use the WRDS API without re-entering your credentials each time. In Google Colab, however, when the file is created it is saved in the temporary home directory (/root/.pgpass) and will be erased whenever you start a new session.  As such you can just respond "`n`".

In [ ]:
db = wrds.Connection()

### Download Data
####Function: `get_crsp_msf_by_ids()`     

**Inputs**
* `db`: WRDS database connection object
* List of identifiers (tickers or PERMNOS)
* `start_date`: a starting date for the data of the form `"yyyy-mm"` (optional)
* `end_date`: and ending data for the data of the form `"yyyy-mm"` (optional)
* If `start_date` and `end_date` are not supplied, the function returns all avaiable data from December 1925 to the present.  

**Output**
* `date`
* `permno`: CRSP permanent security identifier  
* `permco`: CRSP permanent company identifier
* `ticker`
* `comnam`: Company name
* `shrcd`: [Share code](https://wrds-www.wharton.upenn.edu/data-dictionary/form_metadata/crsp_a_stock_msf_identifyinginformation/shrcd/)  
* `exchcd`: [Exchange code](https://wrds-www.wharton.upenn.edu/data-dictionary/form_metadata/crsp_a_stock_msf_identifyinginformation/exchcd/)
* `siccd`:  [Standard Industial Classification Code](https://www.sec.gov/search-filings/standard-industrial-classification-sic-code-list)
* `prc`: A negative price does not indicate that the price fell below zero; rather, it means the price was inferred as the midpoint of the closing inside bid and ask quotes instead of an actual end-of-day trade. When using `prc`, always apply `np.abs(prc)` to ensure the absolute value is used.
* `ret`: Return with dividends
* `retx`: Return without dividends
* `vol`: Volume (\$thousands)
* `shrout`: Shares outstanding (\$thousands)

Identifiers can be either **tickers** or **PERMNOs**. The function will automatically detect the type of identifier you are using. Since tickers can change over time, this becomes an important consideration when pulling data over long periods. To address this, CRSP assigns each security a permanent identifier, the PERMNO, that does not change. In addition, WRDS provides an [online tool](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/tools/translate-to-permcopermno/)
 to translate tickers into PERMCO/PERMNO identifiers. The CRSP PERMCO is a permanent *company* identifier, while the PERMNO is a permanent *security identifier*. Several securities may be associated with the same company since firms can issue various classes of equity.  

In [ ]:
ids=['IBM','F', 'AAPL']    # specify ids
start='2015-01'         # start date
end = '2024-12'         # end date
data=sf.get_crsp_msf_by_ids(db,ids,start,end)
print(data)

### Calculate Turnover
Turnover is calculated as volume during the month divided by shares outstanding at the beginning of the month.  In the CRSP data, `shrout` represents shares outstanding at the *end* of month $t$ and `vol` represents volume over the same month $t$. We therefore create a variable `shrout_lag` and divide volume by `shrout_lag` to calculate turnover.  

In [ ]:
data = data.sort_values(['ticker', 'date']) #sort to make sure lagging works properly
data['shrout_lag'] = data.groupby('ticker')['shrout'].shift(1) # create lagged shrout by ticker
data['turnover'] = data['vol'] / data['shrout_lag'] # compute turnover (volume / lagged shares outstanding)

### Pivot Data
We format format the data using the `pivot` method so we have a dataframe with four columns:  
1. date  
2. IBM Turnover  
3. Ford Turnover  
4. AAPL Turnover  


In [ ]:
# pivot for plotting
pivot_data = data.pivot_table(index=data.index, columns='ticker', values='turnover', aggfunc='first')
pivot_data.index.name = 'date'
print(pivot_data)

### Plot Turnover
We plot each turnover series on the same axis in a loop.  If needed, an AI assitant can prpovide more details and expanation.

In [ ]:
cols = ["AAPL", "F", "IBM"]  # <-- replace with your 4 column names
styles = ['-', '--', ':', '-.'][:len(cols)]  # one style per series

#plots all selected columns on one Axes against the DataFrame’s index.
ax = pivot_data[cols].astype(float).plot(figsize=(10, 6),style=styles)

# set title and axis labels
ax.set_ylabel("Turnover", fontsize=16)
ax.legend(title="Ticker (permno)")
plt.show()